In [1]:
# some standard imports
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import grad as compute_grad   
from datetime import datetime 

#this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

# Gradient descent

In [2]:
# gradient descent function
def gradient_descent(g,w,alpha,max_its,beta,version):    
    # flatten the input function, create gradient based on flat function
    g_flat, unflatten, w = flatten_func(g, w)
    grad = compute_grad(g_flat)

    # record history
    w_hist = []
    w_hist.append(unflatten(w))

    # start gradient descent loop
    z = np.zeros((np.shape(w)))      # momentum term
    
    # over the line
    for k in range(max_its):   
        # plug in value into func and derivative
        grad_eval = grad(w)
        grad_eval.shape = np.shape(w)

        ### normalized or unnormalized descent step? ###
        if version == 'normalized':
            grad_norm = np.linalg.norm(grad_eval)
            if grad_norm == 0:
                grad_norm += 10**-6*np.sign(2*np.random.rand(1) - 1)
            grad_eval /= grad_norm
            
        # take descent step with momentum
        z = beta*z + grad_eval
        w = w - alpha*z

        # record weight update
        w_hist.append(unflatten(w))

    return w_hist

In [3]:
from autograd.misc.flatten import flatten_func
from autograd import grad as compute_grad   

# testing stochastic

In [4]:
# load in large-ish dataset
datapath = '../../mlrefined_datasets/convnet_datasets/feat_face_data.csv'
# datapath = '../../mlrefined_datasets/superlearn_datasets/3d_classification_data_v0.csv'
data = np.loadtxt(datapath,delimiter = ',')

In [7]:
x = data[:,:-1]
y = data[:,-1:]

Nice compact cost function.

In [8]:
# the softmax cost function written more compactly
def softmax(w):
    cost  = np.sum(np.log(1 + np.exp((-y)*(w[0] + np.dot(x,w[1:])))))
    return cost

Compute gradient.

In [9]:
grad = compute_grad(softmax)

Test out cost with an initial weight vector.

In [10]:
scale = 0.1
w_init = scale*np.random.randn(np.shape(x)[1]+1,1)

In [11]:
startTime= datetime.now() 

print (softmax(w_init))

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

6854.28678158
Time elpased (hh:mm:ss.ms) 0:00:00.049834


Compare full gradient to stochastic evaluation.

In [12]:
startTime= datetime.now() 

grad(w_init)

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.069863


Test stochastic.  First the cost.

In [17]:
# 2-class logistic
def stochastic_softmax(w,iter):
    # get subset of points
    x_p = x[iter,:]
    y_p = y[iter]

    # compute cost over just this subset
    cost  = np.sum(np.log(1 + np.exp((-y)*(w[0] + np.dot(x_p,w[1:])))))
    return cost

Evaluate the cost.

In [18]:
startTime= datetime.now() 

p = 10
stochastic_softmax(w_init,p)

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.000710


Now the gradient.

In [19]:
stchastic_grad = compute_grad(stochastic_softmax)

In [20]:
startTime= datetime.now() 
p = 10
stchastic_grad(w_init,p)

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.001331


In [15]:
x_blah = x[10,:]

In [16]:
# 2-class logistic
def stochastic_softmax_2(w):
    # compute cost over just this subset
    cost  = np.sum(np.log(1 + np.exp((-y)*(w[0] + np.dot(x_blah,w[1:])))))
    return cost

In [17]:
startTime= datetime.now() 

stochastic_softmax_2(w_init)

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.000651


In [18]:
stchastic_grad_2 = compute_grad(stochastic_softmax_2)

startTime= datetime.now() 

stchastic_grad_2(w_init)

timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.001948
